In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")

import pickle
import time

import torch

import matplotlib.pyplot as plt
import defmod as dm
import model_graddesc as model
import multimodule_usefulfunctions as mm

torch.set_default_tensor_type(torch.DoubleTensor)


In [34]:
nb_pts1 = 10
nb_pts2 = 7
dim = 2

o1 = torch.tensor([1.,1.])
o2 = torch.tensor([5.,5.])
origin = [o1,o2]
radius = [2.,1.]
numberPoints = [nb_pts1, nb_pts2]
gd_list = mm.multipleCircles(origin, radius, numberPoints)

# Defining the deformation modules
sigma1 = 1.
manifold1 = dm.manifold.Landmarks(dim, nb_pts1, gd=gd_list[0].view(-1))
trans1 = dm.deformationmodules.Translations(manifold1, sigma1)

sigma2 = 20.
manifold2 = dm.manifold.Landmarks(dim, nb_pts2, gd=gd_list[1].view(-1))
trans2 = dm.deformationmodules.GlobalTranslation(manifold2, sigma2)

sigma3 = 1.
rot = dm.deformationmodules.LocalRotation(manifold2, sigma3)
scal = dm.deformationmodules.LocalScaling(manifold2, sigma3)

silent1 = dm.deformationmodules.SilentPoints(manifold1.copy())
silent2 = dm.deformationmodules.SilentPoints(manifold2.copy())

comp1 = dm.deformationmodules.CompoundModule([silent1, trans1])
comp2 = dm.deformationmodules.CompoundModule([silent2, trans2])
module_list = [comp1, comp2]
sigma_bg = 1.
modules = dm.multishape.MultiShapeModule(module_list, sigma_bg)


In [35]:
modules = dm.multishape.MultiShapeModule([comp1, comp2], sigma_background=1., reduce_background=False)
modules.fill_controls_zero()
constr_Id = dm.constraints.Identity_Silent()
#constr_Id = dm.constraints.Identity_Silent_reduced()
H = dm.hamiltonian_multishape.Hamiltonian_multi(modules, constr_Id)

# set parameters
#reg_param = 10**-3 

In [36]:
gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
mom0_tensor = torch.cat(H.module.manifold.unroll_cotan().copy(),0).requires_grad_().view(-1)
    
gd0_tensor.retain_grad()
mom0_tensor.retain_grad()
    
#EnergyFunctional = model.EnergyFunctional(modules, H, constr_Id,source, target, 2, reg_param)

In [37]:
def test_gradcheck_Hamiltonian():
    def hamiltonian(gd, mom):
        
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        H.geodesic_controls()
        
        return H.apply_mom()
    
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), raise_exception=True)

gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(gd.shape)


test_gradcheck_Hamiltonian()

True

In [24]:
def test_gradcheck_autoaction():
    def autoaction(gd, mom):
        mod.manifold.fill_gd(gd)
        mod.manifold.fill_cotan(mom)
        return mod.autoaction()
    return torch.autograd.gradcheck(autoaction, (gd, mom), raise_exception=True)

mod = H.module
gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape)

test_gradcheck_autoaction()

KeyboardInterrupt: 

In [7]:
def test_gradcheck_fields():
    def autoaction(gd, mom):
        mod.manifold.fill_gd(gd)
        mod.manifold.fill_cotan(mom)
        points = torch.tensor([0.,0.]).view(-1,2)
        return mod.field_generator().fields[0](points)
    return torch.autograd.gradcheck(autoaction, (gd, mom), raise_exception=True)

mod = modules
gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape)

test_gradcheck_fields()

True

In [8]:
def test_gradcheck_fields():
    def autoaction(gd, mom):
        mod.manifold.fill_gd(gd)
        mod.manifold.fill_cotan(mom)
        points = torch.tensor([0.,0.]).view(-1,2)
        #return mod.background.field_generator()(points)
        return mod.field_generator().fields[-1](points)
    return torch.autograd.gradcheck(autoaction, (gd, mom), raise_exception=True)

mod = modules
gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape)

test_gradcheck_fields()

True

In [9]:
def test_gradcheck_fields():
    def autoaction(gd, mom):
        mod.manifold.fill_gd(gd)
        mod.manifold.fill_cotan(mom)
        
        fields = mod.field_generator().fieldlist

        gd_bg = mod.manifold.manifold_list[-1].gd_points()[0]
        
        return fields[0](gd_bg)
    
    return torch.autograd.gradcheck(autoaction, (gd, mom), raise_exception=True)

mod = modules
gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape)

test_gradcheck_fields()

True

In [33]:
def test_gradcheck_geocont_from_self():
    def fun(gd, mom):
        mod.manifold.fill_gd(gd)
        mod.manifold.fill_cotan(mom)
        
        man = mod.manifold
        vs = mod.adjoint(man)
        K_q = dm.kernels.K_xx(mod.manifold.gd.view(-1, mod.manifold.dim), mod.sigma)
        x = vs(mod.manifold.gd.view(-1, mod.manifold.dim))
        controls, _ = torch.gesv(x, K_q)
        mod.fill_controls(controls.contiguous().view(-1))
        
        #mod.compute_geodesic_control_from_self(mod.manifold)
        return x, K_q
        #return torch.gesv(x, K_q)
    return torch.autograd.gradcheck(fun, (gd, mom), raise_exception=True)

mod = dm.deformationmodules.Translations(man, sigma=10.)
#mod = H.module

gd0_tensor = torch.cat(mod.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(gd.shape).requires_grad_()

test_gradcheck_geocont_from_self()

True

In [ ]:
def test_gradcheck_H_geodesic_controls():
    def fun(gd, mom):
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        H.geodesic_controls()
        
        return torch.cat([*H.module.controls[0], *H.module.controls[1], *H.module.controls[-1]],0)
        #return H.constraints(H.module)
    
    return torch.autograd.gradcheck(fun, (gd, mom), raise_exception=True)

constr = dm.constraints.Identity_Silent()

mod = modules
gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape)

test_gradcheck_H_geodesic_controls()

In [69]:
def test_gradcheck_apply_constr():
    def hamiltonian(gd, mom):
        
        
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        H.geodesic_controls()
        
        modules.manifold.fill_gd(gd)
        modules.manifold.fill_cotan(mom)
               
        appl_constr = torch.dot(H.module.l.view(-1,1).squeeze(), constr(H.module).view(-1,1).squeeze())
        #return appl_constr
        return H.apply_constr()    
    
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), raise_exception=True)

gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape).requires_grad_()

constr = dm.constraints.Identity_Silent()

test_gradcheck_apply_constr()

True

In [68]:
def test_gradcheck_apply_mom():
    def hamiltonian(gd, mom):
        
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        #H.geodesic_controls()
               
        appl_mom = H.apply_mom()
        return appl_mom
            
    
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), raise_exception=True)

gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor

constr = dm.constraints.Identity_Silent()

test_gradcheck_apply_mom()

True

In [65]:
def test_gradcheck_H_applconstr():
    def fun(gd, mom):
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        H.geodesic_controls()
        
        x = H.module.l.view(-1,1).squeeze()
        y = H.constraints(H.module).view(-1,1).squeeze()
        appl_constr = torch.dot(x, y)

        return torch.dot(x,y)
    
    return torch.autograd.gradcheck(fun, (gd, mom), raise_exception=True)


gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape)

test_gradcheck_H_applconstr()

True

In [19]:
def test_gradcheck_Hamiltonian():
    def hamiltonian(gd, mom):
        
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        H.geodesic_controls()
        
        return H.apply_mom()
    
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), raise_exception=True)

gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(gd.shape)


test_gradcheck_Hamiltonian()

True

In [8]:
H.module.module_list

In [18]:
comp = dm.deformationmodules.CompoundModule([silent, trans1, trans2])
#comp2 = dm.deformationmodules.CompoundModule([trans2])
multi = dm.multishape.MultiShapeModule([comp], sigma_background=0.5)
constr_Id = dm.constraints.Identity_Silent()
H = dm.hamiltonian_multishape.Hamiltonian_multi(multi, constr_Id)

In [58]:
def test_gradcheck_background():
    def hamiltonian(gd, mom):
        
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        
        
        H.module.compute_geodesic_control_from_self(H.module.manifold)

        fields = H.module.field_generator().fieldlist
        constr_mat = constr.constraintsmatrix(H.module)
        
        gd_action = torch.cat([*[torch.cat(man.manifold_list[0].action(mod).unroll_tan()) for mod,man in zip(H.module.module_list[:-1], H.module.manifold)], 
                               torch.cat(H.module.manifold.manifold_list[-1].action(H.module.module_list[-1]).unroll_tan())]).view(-1, H.module.manifold.dim)

        
        B = torch.mm(constr_mat, gd_action.view(-1,1))
        A = torch.mm(torch.mm(constr_mat, H.module.autoaction_silent()), torch.transpose(constr_mat,0,1))
            
        lambda_qp,_ = torch.gesv(B, A)
        H.module.fill_l(lambda_qp)

        tmp = torch.mm(torch.transpose(constr_mat,0,1), lambda_qp)
        
        man = H.module.manifold.copy(retain_grad=True) 
        
        c = 0
        for m in man.manifold_list[:-1]:
            m.manifold_list[0].cotan = m.manifold_list[0].cotan - tmp[c:c+m.manifold_list[0].numel_gd].view(-1)
            c = c+m.manifold_list[0].numel_gd
        for m in man.manifold_list[-1].manifold_list:
            m.cotan = m.cotan - tmp[c:c+m.numel_gd].view(-1)
            c = c+m.numel_gd
        
        H.module.compute_geodesic_control_from_self(man)
        h_qp = H.module.controls
        
                                        
        return lambda_qp, torch.cat(h_qp[0])
        
        
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), raise_exception=True)


gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(gd.shape)

constr = constr_Id


test_gradcheck_background()

True

In [7]:
def test_gradcheck_geocontfromself():
    def hamiltonian(gd, mom):
        #mod = H.module.module_list[0].module_list[1]
        mod.manifold.fill_gd(gd)
        mod.manifold.fill_cotan(mom)
       

        man = mod.manifold.copy()
        mod.compute_geodesic_control_from_self(man)
        return mod.controls
    
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), raise_exception=True)
mod = H.module.module_list[0]

gd0_tensor = torch.cat(modul.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(gd.shape)


test_gradcheck_geocontfromself()

NameError: name 'modul' is not defined

In [6]:
def test_gradcheck_geocontfromself():
    def hamiltonian(gd, mom):
        mod = module
        mod.manifold.fill_gd(gd)
        mod.manifold.fill_cotan(mom)
        
        man = mod.manifold
        mod.compute_geodesic_control_from_self(man)
        return mod.controls
    
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), raise_exception=True)

man = manifold1
module = dm.deformationmodules.Translations(man, sigma=1.)
#module = H.module.module_list[0].module_list[1]
#module = trans1
print(module)
gd0_tensor = torch.cat(module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(gd.shape)


test_gradcheck_geocontfromself()

True

In [43]:
H.module.module_list[0].module_list

In [50]:
sigma1 = 10
manifold1 = dm.manifold.Landmarks(dim, nb_pts1, gd=gd_list[0].view(-1).requires_grad_())
trans1 = dm.deformationmodules.Translations(manifold1, sigma1)